In [7]:
import sys, os, os.path
import librosa
from scipy.io import savemat
import pickle as cPickle
import numpy as np
import glob
import time
import h5py
import numpy as np
import json
import os
import zipfile
import wget
import csv

In [8]:
TRAIN_DIR1 = '/local/slurm-5587015/local/audio/balance_wav/*.wav'
TRAIN_DIR2 = '/local/slurm-5587015/local/audio/unbalanced_wav/*.wav'
VALID_DIR = '/local/slurm-5587015/local/audio/valid_wav/*.wav'
EVAL_DIR = '/local/slurm-5587015/local/audio/eval_wav/*.wav'

In [11]:
list_path = "/jet/home/billyli/data_folder/DayLongAudio/lists/"
meta_path = "/jet/home/billyli/data_folder/DayLongAudio/meta/"
base_path = "/local/slurm-5587015/local/audio"

In [14]:
label_map = {}
with open(os.path.join(meta_path, 'class_labels_indices.csv')) as csv_file:

# label_set = np.loadtxt('./data/class_labels_indices.csv', delimiter=',', dtype='str')
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
#             print(f'\t{row[0]}{row[1]} {row[2]}.')
            label_map[row[1]] = [row[0],row[2]]
            line_count += 1
#     label_set = list(label_set)
#     for i in range(1, len(label_set)):
#         label_map[eval(label_set[i][2])] = label_set[i][0]
print(label_map)

Column names are index, mid, display_name
{'/m/09x0r': ['0', 'Speech'], '/m/05zppz': ['1', 'Male speech, man speaking'], '/m/02zsn': ['2', 'Female speech, woman speaking'], '/m/0ytgt': ['3', 'Child speech, kid speaking'], '/m/01h8n0': ['4', 'Conversation'], '/m/02qldy': ['5', 'Narration, monologue'], '/m/0261r1': ['6', 'Babbling'], '/m/0brhx': ['7', 'Speech synthesizer'], '/m/07p6fty': ['8', 'Shout'], '/m/07q4ntr': ['9', 'Bellow'], '/m/07rwj3x': ['10', 'Whoop'], '/m/07sr1lc': ['11', 'Yell'], '/m/04gy_2': ['12', 'Battle cry'], '/t/dd00135': ['13', 'Children shouting'], '/m/03qc9zr': ['14', 'Screaming'], '/m/02rtxlg': ['15', 'Whispering'], '/m/01j3sz': ['16', 'Laughter'], '/t/dd00001': ['17', 'Baby laughter'], '/m/07r660_': ['18', 'Giggle'], '/m/07s04w4': ['19', 'Snicker'], '/m/07sq110': ['20', 'Belly laugh'], '/m/07rgt08': ['21', 'Chuckle, chortle'], '/m/0463cq4': ['22', 'Crying, sobbing'], '/t/dd00002': ['23', 'Baby cry, infant cry'], '/m/07qz6j3': ['24', 'Whimper'], '/m/07qw_06': ['25

In [38]:
len(label_map)

527

In [41]:
bal_train_wav_list = []
bal_train_dic_by_class ={}


with open(os.path.join(list_path, 'balanced_train_segments.csv')) as bal_csv_file:
    bal_csv_reader = csv.reader(bal_csv_file, delimiter=',')

    bal_count = 0
    missing_count = 0
    for row in bal_csv_reader:
        if bal_count <3:
            print(f'Column names are {", ".join(row)}')
            bal_count += 1
        else:
            wav_path = os.path.join(base_path, 'balance_wav',row[0] +'.wav')
            if os.path.exists(wav_path):
                cur_bal_dict = {"wav": wav_path, "labels": eval(','.join(row[3:]))}
                for label in row[3:]:
                    label = label.replace('"','').strip()
                    if label not in bal_train_dic_by_class:
                        bal_train_dic_by_class[label] = [wav_path]
                        
                    else: 
                        bal_train_dic_by_class[label].append(wav_path)
# #             print(cur_bal_dict)
#                 bal_train_wav_list.append(cur_bal_dict)
                bal_count += 1

            else:
                missing_count+=1
#                 print("missing training: " + row[0])
print(f'balanced count: {bal_count}, missing count: {missing_count}')
# with open('./data/datafiles/audioset_bal_train_data' +'.json', 'w') as f:
#     json.dump({'data': bal_train_wav_list}, f, indent=1)



print('Finished AudioSet Preparation')

Column names are # Segments csv created Sun Mar  5 10:54:31 2017
Column names are # num_ytids=22160,  num_segs=22160,  num_unique_labels=527,  num_positive_labels=52882
Column names are # YTID,  start_seconds,  end_seconds,  positive_labels
balanced count: 21720, missing count: 443
Finished AudioSet Preparation


In [55]:
# print(bal_train_dic_by_class['/t/dd00121'])
ind = 0
for class_id in bal_train_dic_by_class:
    print(len(bal_train_dic_by_class[class_id]))
    for item in bal_train_dic_by_class[class_id][:3]:
        print(label_map[class_id], item)
        corr = librosa.autocorrelate(open(item), max_size=10000)
        print(corr)
    ind +=1
print(ind)

5624
['0', 'Speech'] /local/slurm-5587015/local/audio/balance_wav/--PJHxphWEs.wav


AttributeError: '_io.TextIOWrapper' object has no attribute 'shape'

In [56]:
def extract(wav):
    # Takes a waveform (length 160,000, sampling rate 16,000) and extracts filterbank features (size 400 * 64)
    # If the input is multiple waveforms (i.e. size n * 160,000), extract feature for each waveform (i.e. output size n * 400 * 64)
    if wav.ndim > 1:
        return numpy.stack([extract(w) for w in wav])
    spec = librosa.core.stft(wav, n_fft = 4096,
                             hop_length = 400, win_length = 1024,
                             window = 'hann', center = True, pad_mode = 'constant')
    mel = librosa.feature.melspectrogram(S = numpy.abs(spec), sr = 16000, n_mels = 64, fmax = 8000)
    logmel = librosa.core.power_to_db(mel[:, :400])
    return logmel.T.astype('float32')

In [5]:
hf_audio_train_balanced = h5py.File('/local/slurm-5532397/local/audio_train_balanced.h5', 'w') 
audio_path_balanced = glob.glob(TRAIN_DIR1)
for path in audio_path_balanced:
    hash_id = path.split('/')[-1].split('.')[0]
    wav, _ = librosa.core.load(path, sr = 16000, dtype = "float32")
    feature = extract(wav)
    item_data = hf_audio_train_balanced.create_group(str(hash_id))
    item_data.create_dataset('audio', data=feature)

In [ ]:
hf_audio_train_unbalanced = h5py.File('/local/slurm-5532397/local/audio_train_unbalanced.h5', 'w') 
audio_path_unbalanced = glob.glob(TRAIN_DIR2)
for path in audio_path_unbalanced:
    hash_id = path.split('/')[-1].split('.')[0]
    wav, _ = librosa.core.load(path, sr = 16000, dtype = "float32")
    feature = extract(wav)
    item_data = hf_audio_train_unbalanced.create_group(str(hash_id))
    item_data.create_dataset('audio', data=feature)

In [ ]:
hf_audio_valid = h5py.File('/local/slurm-5532397/local/audio_valid.h5', 'w') 
audio_path_valid = glob.glob(VALID_DIR)
for path in audio_path_valid:
    hash_id = path.split('/')[-1].split('.')[0]
    wav, _ = librosa.core.load(path, sr = 16000, dtype = "float32")
    feature = extract(wav)
    item_data = hf_audio_valid.create_group(str(hash_id))
    item_data.create_dataset('audio', data=feature)

In [ ]:
hf_audio_eval = h5py.File('/local/slurm-5532397/local/audio_eval.h5', 'w') 
audio_path_eval = glob.glob(EVAL_DIR)
for path in audio_path_eval:
    hash_id = path.split('/')[-1].split('.')[0]
    wav, _ = librosa.core.load(path, sr = 16000, dtype = "float32")
    feature = extract(wav)
    item_data = hf_audio_eval.create_group(str(hash_id))
    item_data.create_dataset('audio', data=feature)

In [ ]:
'haha'